# Smart Absensi Berbasis Wajah

Sistem absensi realtime menggunakan face recognition dengan Python.

---

## Fitur Utama:
1. **Registrasi Wajah** - Mendaftarkan wajah baru ke sistem
2. **Absensi Realtime** - Deteksi dan pencatatan absensi otomatis
3. **Visualisasi Data** - Melihat rekap absensi

---


## A. KONFIGURASI AWAL

### Import Library dan Cek Versi


In [15]:
# Import semua library yang diperlukan
import cv2
import face_recognition
import numpy as np
import pandas as pd
import pickle
import os
from datetime import datetime
import time

# Cek versi library
print("=" * 50)
print("VERSI LIBRARY")
print("=" * 50)
print(f"OpenCV: {cv2.__version__}")
print(f"NumPy: {np.__version__}")
print(f"Pandas: {pd.__version__}")
print(f"Face Recognition: {face_recognition.__version__}")
print("=" * 50)
print("✓ Semua library berhasil diimport!")


VERSI LIBRARY
OpenCV: 4.11.0
NumPy: 1.25.2
Pandas: 2.3.3
Face Recognition: 1.2.3
✓ Semua library berhasil diimport!


### Inisialisasi Folder dan File


In [16]:
# Buat folder jika belum ada
folders = ['dataset_wajah', 'encodings', 'output']
for folder in folders:
    if not os.path.exists(folder):
        os.makedirs(folder)
        print(f"✓ Folder '{folder}' dibuat")
    else:
        print(f"✓ Folder '{folder}' sudah ada")

# Buat file absensi.csv jika belum ada
if not os.path.exists('absensi.csv'):
    df = pd.DataFrame(columns=['nama', 'id', 'waktu'])
    df.to_csv('absensi.csv', index=False)
    print("✓ File 'absensi.csv' dibuat")
else:
    print("✓ File 'absensi.csv' sudah ada")

print("\n✓ Inisialisasi selesai!")


✓ Folder 'dataset_wajah' sudah ada
✓ Folder 'encodings' sudah ada
✓ Folder 'output' sudah ada
✓ File 'absensi.csv' dibuat

✓ Inisialisasi selesai!


### Test Kamera

Fungsi untuk mencari kamera yang tersedia dan menguji apakah kamera berfungsi.


In [17]:
def find_camera():
    """
    Mencari kamera yang tersedia dengan mencoba index 0, 1, dan 2
    Returns: camera index yang berfungsi atau None
    """
    for i in range(3):
        cap = cv2.VideoCapture(i)
        if cap.isOpened():
            print(f"✓ Kamera ditemukan di index {i}")
            return i, cap
        cap.release()
    return None, None

# Test kamera
print("Mencari kamera yang tersedia...")
camera_index, test_cap = find_camera()

if test_cap is None:
    print("\n❌ ERROR: Tidak ada kamera yang terdeteksi!")
    print("Solusi:")
    print("1. Pastikan kamera laptop berfungsi")
    print("2. Pastikan tidak ada aplikasi lain yang menggunakan kamera")
    print("3. Coba restart kernel dan jalankan ulang")
else:
    print("\nTekan 'q' untuk keluar dari preview kamera...\n")
    print("Membuka preview kamera dalam 2 detik...")
    time.sleep(2)
    
    # Tampilkan preview kamera selama 5 detik atau sampai 'q' ditekan
    start_time = time.time()
    frame_count = 0
    
    while True:
        ret, frame = test_cap.read()
        if not ret:
            print("❌ Gagal membaca frame dari kamera")
            break
        
        frame_count += 1
        
        # Tambahkan teks ke frame
        cv2.putText(frame, "TEST KAMERA - Tekan 'q' untuk keluar", 
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(frame, f"Frame: {frame_count}", 
                    (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        
        cv2.imshow('Test Kamera', frame)
        
        # Keluar jika 'q' ditekan atau sudah 10 detik
        if cv2.waitKey(1) & 0xFF == ord('q') or (time.time() - start_time) > 10:
            break
    
    # Tutup kamera dengan benar
    test_cap.release()
    cv2.destroyAllWindows()
    
    # Tunggu sebentar untuk memastikan window tertutup
    for i in range(4):
        cv2.waitKey(1)
    
    print(f"\n✓ Test kamera selesai! ({frame_count} frame berhasil dibaca)")
    print(f"✓ Kamera berfungsi dengan baik di index {camera_index}")


Mencari kamera yang tersedia...
✓ Kamera ditemukan di index 0

Tekan 'q' untuk keluar dari preview kamera...

Membuka preview kamera dalam 2 detik...

✓ Test kamera selesai! (254 frame berhasil dibaca)
✓ Kamera berfungsi dengan baik di index 0


---

## B. MODUL REGISTRASI WAJAH

### Fungsi Registrasi Wajah Baru


In [18]:
def registrasi_wajah(nama, id_mahasiswa, jumlah_foto_minimal=3):
    """
    Registrasi wajah baru ke sistem dengan MULTIPLE ENCODINGS
    Minimum 3 foto diperlukan untuk akurasi optimal
    
    Parameters:
    - nama: Nama lengkap (string)
    - id_mahasiswa: NIM atau ID unik (string)
    - jumlah_foto_minimal: Jumlah foto minimal yang harus diambil (default: 3)
    
    Returns:
    - True jika berhasil, False jika gagal
    """
    
    print("=" * 60)
    print("REGISTRASI WAJAH (MULTIPLE ENCODINGS)")
    print("=" * 60)
    print(f"Nama: {nama}")
    print(f"ID: {id_mahasiswa}")
    print(f"Jumlah foto minimal: {jumlah_foto_minimal} foto")
    print("\n⚠ PENTING: Minimum 3 foto diperlukan untuk absensi!")
    print("\nInstruksi:")
    print("1. Pastikan wajah Anda terlihat jelas di kamera")
    print("2. Tekan 'c' untuk capture foto (minimal 3 kali)")
    print("3. Variasikan sedikit posisi/ekspresi untuk setiap foto")
    print("4. Tekan 'q' untuk keluar atau batalkan\n")
    
    # Cari kamera
    camera_index, cap = find_camera()
    if cap is None:
        print("❌ Kamera tidak dapat diakses")
        return False
    
    # List untuk menyimpan semua encoding dan foto
    all_encodings = []
    foto_count = 0
    
    # Reset error flag untuk memungkinkan error ditampilkan lagi
    if hasattr(registrasi_wajah, '_error_printed'):
        delattr(registrasi_wajah, '_error_printed')
    
    print("Membuka kamera...\n")
    time.sleep(1)
    
    # Skip beberapa frame awal untuk stabilisasi kamera
    for _ in range(5):
        cap.read()
    
    while foto_count < jumlah_foto_minimal:
        ret, frame = cap.read()
        if not ret:
            print("❌ Gagal membaca frame")
            break
        
        # Inisialisasi face_locations
        face_locations = []
        rgb_frame = None
        
        try:
            # Gunakan pendekatan yang SAMA PERSIS dengan absensi_realtime yang berfungsi
            # Resize frame untuk performa (0.25x seperti di absensi_realtime)
            small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
            
            # Konversi BGR ke RGB - SAMA PERSIS dengan absensi_realtime
            rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
            
            # Deteksi wajah - SAMA PERSIS dengan absensi_realtime (tanpa PIL, tanpa validasi tambahan)
            # Jika absensi_realtime berfungsi dengan cara ini, registrasi juga harusnya berfungsi
            face_locations_small = face_recognition.face_locations(rgb_small_frame)
            
            # Scale kembali lokasi wajah ke ukuran frame asli (kalikan dengan 4 karena resize 0.25x)
            face_locations = []
            for (top, right, bottom, left) in face_locations_small:
                face_locations.append((top * 4, right * 4, bottom * 4, left * 4))
            
            # Simpan rgb_frame untuk encoding nanti (gunakan frame asli untuk kualitas lebih baik)
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # Pastikan format yang benar untuk encoding
            if not rgb_frame.flags['C_CONTIGUOUS']:
                rgb_frame = np.ascontiguousarray(rgb_frame)
            if rgb_frame.dtype != np.uint8:
                rgb_frame = rgb_frame.astype(np.uint8)
                
        except cv2.error as e:
            # Error dari OpenCV
            face_locations = []
            rgb_frame = None
            if not hasattr(registrasi_wajah, '_error_printed'):
                print(f"⚠ Error OpenCV: {e}")
                print(f"   Frame shape: {frame.shape if frame is not None else 'None'}")
                print(f"   Frame dtype: {frame.dtype if frame is not None else 'None'}")
                registrasi_wajah._error_printed = True
        except (ValueError, AssertionError) as e:
            # Error validasi
            face_locations = []
            rgb_frame = None
            if not hasattr(registrasi_wajah, '_error_printed'):
                print(f"⚠ Error validasi: {e}")
                registrasi_wajah._error_printed = True
        except RuntimeError as e:
            # Error dari face_recognition
            face_locations = []
            rgb_frame = None
            if not hasattr(registrasi_wajah, '_error_printed'):
                print(f"⚠ Error Runtime dari face_recognition: {e}")
                print(f"   Frame shape: {frame.shape if 'frame' in locals() else 'N/A'}")
                print(f"   Frame dtype: {frame.dtype if 'frame' in locals() else 'N/A'}")
                print(f"   NumPy version: {np.__version__}")
                if hasattr(np, '__version__') and int(np.__version__.split('.')[0]) >= 2:
                    print("   ⚠ NumPy 2.x mungkin tidak kompatibel!")
                    print("   Coba downgrade: pip install numpy==1.23.5")
                registrasi_wajah._error_printed = True
        except Exception as e:
            # Error lainnya
            face_locations = []
            rgb_frame = None
            # Hanya print error sekali untuk menghindari spam
            if not hasattr(registrasi_wajah, '_error_printed'):
                print(f"⚠ Error dalam deteksi wajah: {e}")
                print(f"   Error type: {type(e).__name__}")
                registrasi_wajah._error_printed = True
        
        # Gambar kotak di sekitar wajah
        display_frame = frame.copy()
        for (top, right, bottom, left) in face_locations:
            cv2.rectangle(display_frame, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.putText(display_frame, "Wajah Terdeteksi", (left, top - 10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        
        # Status dengan progress
        if len(face_locations) > 0:
            status_text = f"STATUS: Wajah terdeteksi - Tekan 'c' untuk capture ({foto_count + 1}/{jumlah_foto_minimal})"
            color = (0, 255, 0)
        else:
            status_text = f"STATUS: Posisikan wajah di depan kamera ({foto_count + 1}/{jumlah_foto_minimal})"
            color = (0, 0, 255)
        
        # Tips untuk variasi foto
        if foto_count == 0:
            tip = "Foto 1: Wajah lurus, ekspresi netral"
        elif foto_count == 1:
            tip = "Foto 2: Sedikit miring kiri/kanan"
        elif foto_count == 2:
            tip = "Foto 3: Sedikit miring kanan/kiri atau ekspresi berbeda"
        else:
            tip = f"Foto {foto_count + 1}: Variasi posisi/ekspresi"
        
        cv2.putText(display_frame, status_text, (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        cv2.putText(display_frame, f"Nama: {nama} | ID: {id_mahasiswa}", (10, 60),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        cv2.putText(display_frame, tip, (10, 90),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)
        cv2.putText(display_frame, "'c' = capture | 'q' = keluar", (10, display_frame.shape[0] - 10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        cv2.imshow('Registrasi Wajah', display_frame)
        
        key = cv2.waitKey(1) & 0xFF
        
        # Capture foto
        if key == ord('c'):
            if len(face_locations) > 0:
                # Simpan foto dengan nomor urut
                filename = f"{nama}_{id_mahasiswa}_{foto_count + 1}.jpg"
                filepath = os.path.join('dataset_wajah', filename)
                cv2.imwrite(filepath, frame)
                print(f"\n✓ Foto {foto_count + 1} berhasil disimpan: {filepath}")
                
                # Encode wajah
                print("Membuat encoding wajah...")
                if rgb_frame is None:
                    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    if not rgb_frame.flags['C_CONTIGUOUS']:
                        rgb_frame = np.ascontiguousarray(rgb_frame)
                    if rgb_frame.dtype != np.uint8:
                        rgb_frame = rgb_frame.astype(np.uint8)
                
                try:
                    face_encoding = face_recognition.face_encodings(rgb_frame, face_locations)[0]
                    all_encodings.append(face_encoding)
                    foto_count += 1
                    print(f"✓ Encoding {foto_count} berhasil dibuat")
                    
                    if foto_count < jumlah_foto_minimal:
                        print(f"  Ambil {jumlah_foto_minimal - foto_count} foto lagi...")
                        print("  (Variasi sedikit posisi/ekspresi untuk hasil lebih baik)\n")
                        time.sleep(1.5)  # Pause sebelum foto berikutnya
                    else:
                        print("\n✓ Semua foto minimal berhasil diambil!")
                        break
                except Exception as e:
                    print(f"❌ Error saat encoding: {e}")
            else:
                print("\n⚠ Tidak ada wajah yang terdeteksi. Posisikan wajah dengan benar.")
        
        # Keluar
        elif key == ord('q'):
            if foto_count > 0:
                print(f"\n⚠ Anda sudah mengambil {foto_count} foto dari {jumlah_foto_minimal} yang diperlukan.")
                print("⚠ Registrasi tidak akan tersimpan jika kurang dari minimum!")
                konfirmasi = input("Yakin ingin keluar? (y/n): ")
                if konfirmasi.lower() != 'y':
                    continue
            print("\nRegistrasi dibatalkan.")
            break
    
    # Tutup kamera
    cap.release()
    cv2.destroyAllWindows()
    for i in range(4):
        cv2.waitKey(1)
    
    # Simpan semua encoding jika memenuhi minimum
    if len(all_encodings) >= jumlah_foto_minimal:
        encoding_data = {
            'nama': nama,
            'id': id_mahasiswa,
            'encodings': all_encodings,  # ARRAY of encodings
            'jumlah_encoding': len(all_encodings),
            'tanggal_registrasi': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'format': 'multiple'
        }
        
        encoding_filename = f"{nama}_{id_mahasiswa}.pickle"
        encoding_filepath = os.path.join('encodings', encoding_filename)
        
        with open(encoding_filepath, 'wb') as f:
            pickle.dump(encoding_data, f)
        
        print(f"\n✓ {len(all_encodings)} encoding berhasil disimpan: {encoding_filepath}")
        print("=" * 60)
        print("✓ REGISTRASI BERHASIL!")
        print("=" * 60)
        print(f"✓ {len(all_encodings)} foto dan encoding tersimpan")
        print("✓ Anda sekarang dapat melakukan absensi!")
        print("=" * 60)
        return True
    else:
        print("\n" + "=" * 60)
        print("❌ REGISTRASI GAGAL!")
        print("=" * 60)
        print(f"⚠ Hanya {len(all_encodings)} foto yang berhasil diambil")
        print(f"⚠ Minimum {jumlah_foto_minimal} foto diperlukan untuk absensi")
        print("⚠ Data tidak disimpan. Silakan registrasi ulang dengan minimal 3 foto.")
        print("=" * 60)
        return False

print("✓ Fungsi registrasi_wajah (MULTIPLE ENCODINGS) siap digunakan!")


✓ Fungsi registrasi_wajah (MULTIPLE ENCODINGS) siap digunakan!


### Proses Registrasi Wajah dan Identitas

**Jalankan cell di bawah untuk mendaftarkan wajah baru:**


In [20]:
# CONTOH REGISTRASI - Edit nama dan ID sesuai kebutuhan

nama_mahasiswa = input("Masukkan Nama: ")
nim_mahasiswa = input("Masukkan NIM/ID: ")

# Validasi input
if nama_mahasiswa.strip() == "" or nim_mahasiswa.strip() == "":
    print("❌ Nama dan ID tidak boleh kosong!")
else:
    hasil = registrasi_wajah(nama_mahasiswa, nim_mahasiswa)
    if hasil:
        print("\n✓ Registrasi selesai! Wajah berhasil didaftarkan ke sistem.")
    else:
        print("\n❌ Registrasi gagal. Silakan coba lagi.")


Masukkan Nama:  jsd
Masukkan NIM/ID:  daw


REGISTRASI WAJAH (MULTIPLE ENCODINGS)
Nama: jsd
ID: daw
Jumlah foto minimal: 3 foto

⚠ PENTING: Minimum 3 foto diperlukan untuk absensi!

Instruksi:
1. Pastikan wajah Anda terlihat jelas di kamera
2. Tekan 'c' untuk capture foto (minimal 3 kali)
3. Variasikan sedikit posisi/ekspresi untuk setiap foto
4. Tekan 'q' untuk keluar atau batalkan

✓ Kamera ditemukan di index 0
Membuka kamera...


Registrasi dibatalkan.

❌ REGISTRASI GAGAL!
⚠ Hanya 0 foto yang berhasil diambil
⚠ Minimum 3 foto diperlukan untuk absensi
⚠ Data tidak disimpan. Silakan registrasi ulang dengan minimal 3 foto.

❌ Registrasi gagal. Silakan coba lagi.


---

## C. MODUL ABSENSI WAJAH (REALTIME)

### Load Semua Encoding Wajah yang Terdaftar


In [21]:
def load_encodings(min_encoding_per_orang=3):
    """
    Load semua encoding wajah dari folder encodings/
    Support untuk format lama (single encoding) dan format baru (multiple encodings)
    Validasi minimum encoding per orang untuk absensi
    
    Parameters:
    - min_encoding_per_orang: Minimum encoding per orang untuk valid (default: 3)
    
    Returns:
    - known_encodings: list of face encodings (bisa multiple per orang)
    - known_data: list of dictionaries {'nama': ..., 'id': ...}
    """
    known_encodings = []
    known_data = []
    
    encoding_files = [f for f in os.listdir('encodings') if f.endswith('.pickle')]
    
    if len(encoding_files) == 0:
        print("⚠ Tidak ada encoding yang ditemukan!")
        print("Silakan registrasi wajah terlebih dahulu.")
        return [], []
    
    print(f"Loading {len(encoding_files)} file encoding...")
    
    total_encodings = 0
    valid_count = 0
    invalid_count = 0
    
    for filename in encoding_files:
        filepath = os.path.join('encodings', filename)
        try:
            with open(filepath, 'rb') as f:
                data = pickle.load(f)
                
                # Format baru: multiple encodings
                if 'encodings' in data and isinstance(data['encodings'], list):
                    encodings = data['encodings']
                    jumlah = len(encodings)
                    
                    # Validasi minimum encoding
                    if jumlah >= min_encoding_per_orang:
                        known_encodings.extend(encodings)
                        for i in range(jumlah):
                            known_data.append({
                                'nama': data['nama'],
                                'id': data['id'],
                                'encoding_index': i + 1,
                                'total_encodings': jumlah
                            })
                        total_encodings += jumlah
                        valid_count += 1
                        print(f"  ✓ {data['nama']} ({data['id']}) - {jumlah} encoding [VALID]")
                    else:
                        invalid_count += 1
                        print(f"  ⚠ {data['nama']} ({data['id']}) - {jumlah} encoding [TIDAK VALID - < {min_encoding_per_orang}]")
                
                # Format lama: single encoding (backward compatible, tapi tidak valid untuk absensi)
                elif 'encoding' in data:
                    invalid_count += 1
                    print(f"  ⚠ {data['nama']} ({data['id']}) - 1 encoding [FORMAT LAMA - Perlu registrasi ulang]")
                
                else:
                    invalid_count += 1
                    print(f"  ❌ {filename}: Format tidak dikenali")
                    
        except Exception as e:
            invalid_count += 1
            print(f"  ❌ Error loading {filename}: {e}")
    
    print(f"\n✓ Total {total_encodings} encoding dari {valid_count} orang VALID berhasil di-load!")
    if invalid_count > 0:
        print(f"⚠ {invalid_count} orang tidak valid (perlu registrasi ulang dengan minimal {min_encoding_per_orang} foto)")
    
    if len(known_encodings) == 0:
        print("\n❌ Tidak ada encoding yang valid untuk absensi!")
        print(f"   Silakan registrasi wajah dengan minimal {min_encoding_per_orang} foto per orang.")
    
    return known_encodings, known_data

print("✓ Fungsi load_encodings (MULTIPLE ENCODINGS + VALIDASI) siap digunakan!")


✓ Fungsi load_encodings (MULTIPLE ENCODINGS + VALIDASI) siap digunakan!


In [22]:
def absensi_realtime():
    """
    Jalankan sistem absensi realtime dengan deteksi wajah (OPTIMASI MULTIPLE ENCODINGS)
    - Bingkai tidak kedip-kedip (tracking posisi)
    - Foto hanya disimpan sekali saat pertama kali terdeteksi
    - Optimasi performa untuk multiple encodings
    - Smooth dan tidak lag
    """
    import threading
    from queue import Queue
    
    print("=" * 60)
    print("SISTEM ABSENSI REALTIME (OPTIMASI MULTIPLE ENCODINGS)")
    print("=" * 60)
    
    # Load encoding dengan validasi minimum
    known_encodings, known_data = load_encodings(min_encoding_per_orang=3)
    
    if len(known_encodings) == 0:
        print("\n❌ Tidak ada encoding yang valid untuk absensi!")
        print("   Silakan registrasi wajah dengan minimal 3 foto per orang.")
        return
    
    # ========== OPTIMASI: Pre-compute array NumPy sekali ==========
    known_encodings_array = np.array(known_encodings)
    
    # ========== OPTIMASI: Group encodings per orang untuk tracking ==========
    # Buat mapping: person_id -> list of encoding indices
    person_encoding_map = {}
    for idx, person_data in enumerate(known_data):
        person_id = person_data['id']
        if person_id not in person_encoding_map:
            person_encoding_map[person_id] = []
        person_encoding_map[person_id].append(idx)
    
    total_orang = len(person_encoding_map)
    total_encodings = len(known_encodings)
    avg_encoding = total_encodings / total_orang if total_orang > 0 else 0
    
    print(f"Total encoding: {total_encodings} dari {total_orang} orang")
    print(f"Rata-rata: {avg_encoding:.1f} encoding/orang")
    
    # Cari kamera
    camera_index, cap = find_camera()
    if cap is None:
        print("❌ Kamera tidak dapat diakses")
        return
    
    # Set untuk tracking siapa yang sudah absen di sesi ini
    sudah_absen = set()
    
    # ========== FIX: Tracking untuk bingkai tidak kedip-kedip ==========
    # Simpan posisi dan info wajah terakhir yang terdeteksi
    last_detected_faces = []  # List of {'person_id': ..., 'box': (top, right, bottom, left), 'nama': ..., 'confidence': ...}
    # ===================================================================
    
    # Queue untuk batch I/O operations (threading)
    save_queue = Queue()
    
    def save_worker():
        """Thread worker untuk menyimpan data secara asynchronous"""
        df_absensi = pd.read_csv('absensi.csv')
        while True:
            item = save_queue.get()
            if item is None:  # Signal untuk stop
                if len(df_absensi) > 0:
                    df_absensi.to_csv('absensi.csv', index=False)
                break
            
            new_row = pd.DataFrame([{
                'nama': item['nama'],
                'id': item['id'],
                'waktu': item['waktu']
            }])
            df_absensi = pd.concat([df_absensi, new_row], ignore_index=True)
            
            # ========== FIX: Hanya simpan foto sekali ==========
            cv2.imwrite(item["output_path"], item["frame"])
            # ===================================================
            
            if save_queue.qsize() == 0 or len(df_absensi) % 3 == 0:
                df_absensi.to_csv('absensi.csv', index=False)
            
            save_queue.task_done()
    
    io_thread = threading.Thread(target=save_worker, daemon=True)
    io_thread.start()
    
    print("\nInstruksi:")
    print("- Posisikan wajah di depan kamera")
    print("- Sistem akan otomatis mendeteksi dan mencatat absensi")
    print("- Maksimal 2 wajah diproses sekaligus untuk performa optimal")
    print("- Tekan 'q' untuk keluar\n")
    print("Membuka kamera...\n")
    time.sleep(2)
    
    # ========== PARAMETER OPTIMASI DINAMIS ==========
    frame_count = 0
    
    # Optimasi dinamis berdasarkan jumlah encoding
    if total_encodings <= 20:
        # Sedikit encoding: bisa lebih agresif
        process_every_n_frames = 3  # Process lebih sering
        resize_factor = 0.25  # Resolusi lebih tinggi
    elif total_encodings <= 100:
        # Sedang: setting default
        process_every_n_frames = 4  # Default
        resize_factor = 0.2  # Default
    else:
        # Banyak encoding: lebih agresif skip
        process_every_n_frames = 5  # Skip lebih banyak
        resize_factor = 0.15  # Resolusi lebih rendah
    
    use_hog_model = True  # Gunakan HOG model (lebih cepat)
    max_faces = 2  # Limit maksimal 2 wajah
    frame_timeout = 10  # FIX: Hapus bingkai setelah 10 frame tidak terdeteksi
    
    print(f"⚙️  Optimasi: Process setiap {process_every_n_frames} frame, Resize {resize_factor*100:.0f}%")
    print(f"⚙️  Total encoding: {total_encodings}\n")
    # ========================================
    
    frame_without_detection = 0  # Counter untuk menghapus bingkai lama
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("❌ Gagal membaca frame")
            break
        
        frame_count += 1
        
        # ========== FIX: Gambar bingkai dari tracking di SEMUA frame ==========
        # Ini membuat bingkai tidak kedip-kedip
        current_detected_faces = []
        scale_factor = int(1 / resize_factor)
        
        # Gambar bingkai dari deteksi terakhir
        for face_info in last_detected_faces:
            top, right, bottom, left = face_info['box']
            nama = face_info['nama']
            person_id = face_info['person_id']
            confidence = face_info.get('confidence', 0)
            
            # Gambar kotak dan nama (selalu muncul di semua frame)
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 255, 0), cv2.FILLED)
            cv2.putText(frame, f"{nama} ({person_id})", (left + 6, bottom - 6),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        # ======================================================================
        
        # Process hanya setiap n frame (skip lebih banyak untuk performa)
        if frame_count % process_every_n_frames == 0:
            # Resize untuk performa
            small_frame = cv2.resize(frame, (0, 0), fx=resize_factor, fy=resize_factor)
            rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
            
            # Deteksi wajah dengan HOG model (lebih cepat)
            if use_hog_model:
                face_locations = face_recognition.face_locations(rgb_small_frame, model='hog')
            else:
                face_locations = face_recognition.face_locations(rgb_small_frame)
            
            # OPTIMASI: Limit maksimal wajah yang diproses
            if len(face_locations) > max_faces:
                face_areas = [(bottom - top) * (right - left)
                              for (top, right, bottom, left) in face_locations]
                sorted_indices = sorted(range(len(face_areas)),
                                       key=lambda i: face_areas[i], reverse=True)
                face_locations = [face_locations[i] for i in sorted_indices[:max_faces]]
            
            # Encoding wajah yang terdeteksi
            if len(face_locations) > 0:
                face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
                frame_without_detection = 0  # Reset counter
                
                # Loop untuk setiap wajah yang terdeteksi
                for face_encoding, face_location in zip(face_encodings, face_locations):
                    # ========== OPTIMASI: compare_faces sudah dioptimasi NumPy ==========
                    # Operasi ini sangat cepat meskipun dengan banyak encoding
                    matches = face_recognition.compare_faces(
                        known_encodings_array, 
                        face_encoding, 
                        tolerance=0.6
                    )
                    face_distances = face_recognition.face_distance(
                        known_encodings_array, 
                        face_encoding
                    )
                    best_match_index = np.argmin(face_distances)
                    
                    if matches[best_match_index]:
                        person_data = known_data[best_match_index]
                        nama = person_data['nama']
                        person_id = person_data['id']
                        confidence = (1 - face_distances[best_match_index]) * 100
                        
                        # ========== OPTIMASI: Cari encoding terbaik dari orang yang sama ==========
                        # Jika orang ini punya multiple encodings, cari yang terbaik
                        if person_id in person_encoding_map and len(person_encoding_map[person_id]) > 1:
                            # Cari encoding dengan confidence tertinggi dari orang yang sama
                            person_indices = person_encoding_map[person_id]
                            person_distances = [face_distances[i] for i in person_indices]
                            best_person_index = person_indices[np.argmin(person_distances)]
                            best_person_distance = min(person_distances)
                            confidence = (1 - best_person_distance) * 100
                            
                            # Update person_data dengan yang terbaik
                            person_data = known_data[best_person_index]
                            nama = person_data['nama']
                            person_id = person_data['id']
                        
                        # Scale kembali lokasi wajah
                        top, right, bottom, left = face_location
                        top *= scale_factor
                        right *= scale_factor
                        bottom *= scale_factor
                        left *= scale_factor
                        
                        # ========== FIX: Update tracking dengan posisi baru ==========
                        # Simpan info wajah untuk ditampilkan di frame berikutnya
                        current_detected_faces.append({
                            'person_id': person_id,
                            'box': (top, right, bottom, left),
                            'nama': nama,
                            'confidence': confidence
                        })
                        # ============================================================
                        
                        # ========== FIX: Catat absensi HANYA jika belum absen ==========
                        if person_id not in sudah_absen:
                            waktu_sekarang = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                            output_filename = f"{nama}_{person_id}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.jpg"
                            output_path = os.path.join('output', output_filename)
                            
                            # OPTIMASI: Async I/O dengan queue (non-blocking)
                            # HANYA simpan sekali saat pertama kali terdeteksi
                            save_queue.put({
                                'nama': nama,
                                'id': person_id,
                                'waktu': waktu_sekarang,
                                'frame': frame.copy(),  # Copy frame untuk thread
                                'output_path': output_path
                            })
                            
                            # Tandai sudah absen (PENTING: ini mencegah save berulang)
                            sudah_absen.add(person_id)
                            
                            print(f"\n{'='*60}")
                            print(f"✓ ABSENSI TERCATAT")
                            print(f"{'='*60}")
                            print(f"Nama       : {nama}")
                            print(f"ID         : {person_id}")
                            print(f"Waktu      : {waktu_sekarang}")
                            print(f"Confidence : {confidence:.2f}%")
                            print(f"Foto       : {output_path}")
                            print(f"{'='*60}\n")
                    else:
                        # Wajah tidak dikenali - tidak perlu tracking
                        top, right, bottom, left = face_location
                        top *= scale_factor
                        right *= scale_factor
                        bottom *= scale_factor
                        left *= scale_factor
                        
                        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
                        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
                        cv2.putText(frame, "Tidak Dikenali", (left + 6, bottom - 6),
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                
                # ========== FIX: Update tracking dengan wajah yang baru terdeteksi ==========
                last_detected_faces = current_detected_faces
                # ===========================================================================
            else:
                # Tidak ada wajah terdeteksi
                frame_without_detection += 1
                # Hapus bingkai lama setelah beberapa frame tidak terdeteksi
                if frame_without_detection > frame_timeout:
                    last_detected_faces = []
        
        # Tambahkan info ke frame
        estimated_fps = 30 // process_every_n_frames
        cv2.putText(frame, "SISTEM ABSENSI - Tekan 'q' untuk keluar", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.putText(frame, f"Sudah Absen: {len(sudah_absen)} orang", (10, 60),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        cv2.putText(frame, f"FPS: ~{estimated_fps} | Encoding: {total_encodings} | Max: {max_faces} wajah", (10, 90),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)
        
        cv2.imshow('Absensi Realtime', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Stop worker thread
    save_queue.put(None)
    io_thread.join(timeout=2)  # Tunggu max 2 detik untuk selesai
    
    # Tutup kamera
    cap.release()
    cv2.destroyAllWindows()
    for i in range(4):
        cv2.waitKey(1)
    
    print("\n" + "=" * 60)
    print("SESI ABSENSI SELESAI")
    print("=" * 60)
    print(f"Total yang hadir: {len(sudah_absen)} orang")
    if len(sudah_absen) > 0:
        print("\nDaftar yang hadir:")
        for person_id in sudah_absen:
            for data in known_data:
                if data['id'] == person_id:
                    print(f"  - {data['nama']} ({person_id})")
    print("=" * 60)

print("✓ Fungsi absensi_realtime (OPTIMASI MULTIPLE ENCODINGS) siap digunakan!")

✓ Fungsi absensi_realtime (OPTIMASI MULTIPLE ENCODINGS) siap digunakan!


### Jalankan Absensi Realtime

**Jalankan cell di bawah untuk memulai sistem absensi realtime:**


In [23]:
# JALANKAN ABSENSI REALTIME
absensi_realtime()


SISTEM ABSENSI REALTIME (OPTIMASI MULTIPLE ENCODINGS)
Loading 2 file encoding...
  ✓ Fahren Andrean Rangkuti (23215030) - 3 encoding [VALID]
  ⚠ Fahren (23215030) - 1 encoding [FORMAT LAMA - Perlu registrasi ulang]

✓ Total 3 encoding dari 1 orang VALID berhasil di-load!
⚠ 1 orang tidak valid (perlu registrasi ulang dengan minimal 3 foto)
Total encoding: 3 dari 1 orang
Rata-rata: 3.0 encoding/orang
✓ Kamera ditemukan di index 0

Instruksi:
- Posisikan wajah di depan kamera
- Sistem akan otomatis mendeteksi dan mencatat absensi
- Maksimal 2 wajah diproses sekaligus untuk performa optimal
- Tekan 'q' untuk keluar

Membuka kamera...

⚙️  Optimasi: Process setiap 3 frame, Resize 25%
⚙️  Total encoding: 3


✓ ABSENSI TERCATAT
Nama       : Fahren Andrean Rangkuti
ID         : 23215030
Waktu      : 2025-11-29 19:44:08
Confidence : 68.77%
Foto       : output\Fahren Andrean Rangkuti_23215030_20251129_194408.jpg


SESI ABSENSI SELESAI
Total yang hadir: 1 orang

Daftar yang hadir:
  - Fahren Andr

---

## D. VISUALISASI DATA ABSENSI

### Lihat Data Absensi


In [24]:
# Load data absensi
df_absensi = pd.read_csv('absensi.csv')

if len(df_absensi) == 0:
    print("⚠ Belum ada data absensi.")
else:
    print("=" * 80)
    print("DATA ABSENSI")
    print("=" * 80)
    print(f"\nTotal Records: {len(df_absensi)}\n")
    display(df_absensi)


DATA ABSENSI

Total Records: 1



,nama,id,waktu
0,Fahren Andrean Rangkuti,23215030,2025-11-29 19:44:08


### Statistik Absensi


In [25]:
if len(df_absensi) > 0:
    print("=" * 80)
    print("STATISTIK ABSENSI")
    print("=" * 80)
    
    # Hitung jumlah absensi per orang
    absensi_per_orang = df_absensi.groupby(['nama', 'id']).size().reset_index(name='jumlah_absensi')
    absensi_per_orang = absensi_per_orang.sort_values('jumlah_absensi', ascending=False)
    
    print("\n1. Jumlah Absensi per Orang:")
    print("=" * 80)
    display(absensi_per_orang)
    
    # Absensi terbaru
    print("\n2. 10 Absensi Terbaru:")
    print("=" * 80)
    display(df_absensi.tail(10))
    
    # Ringkasan
    print("\n3. Ringkasan:")
    print("=" * 80)
    print(f"Total Absensi      : {len(df_absensi)} kali")
    print(f"Jumlah Orang Unik  : {df_absensi['id'].nunique()} orang")
    print(f"Absensi Pertama    : {df_absensi['waktu'].min() if len(df_absensi) > 0 else '-'}")
    print(f"Absensi Terakhir   : {df_absensi['waktu'].max() if len(df_absensi) > 0 else '-'}")
    print("=" * 80)
else:
    print("⚠ Belum ada data untuk ditampilkan.")


STATISTIK ABSENSI

1. Jumlah Absensi per Orang:


,nama,id,jumlah_absensi
0,Fahren Andrean Rangkuti,23215030,1



2. 10 Absensi Terbaru:


,nama,id,waktu
0,Fahren Andrean Rangkuti,23215030,2025-11-29 19:44:08



3. Ringkasan:
Total Absensi      : 1 kali
Jumlah Orang Unik  : 1 orang
Absensi Pertama    : 2025-11-29 19:44:08
Absensi Terakhir   : 2025-11-29 19:44:08


### Export Data ke Excel (Opsional)


In [26]:
# Export ke Excel jika ada data
if len(df_absensi) > 0:
    try:
        output_excel = f"laporan_absensi_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
        df_absensi.to_excel(output_excel, index=False)
        print(f"✓ Data berhasil di-export ke: {output_excel}")
    except:
        print("⚠ Gagal export ke Excel. Install openpyxl dengan: pip install openpyxl")
        # Export ke CSV sebagai alternatif
        output_csv = f"laporan_absensi_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        df_absensi.to_csv(output_csv, index=False)
        print(f"✓ Data berhasil di-export ke CSV: {output_csv}")
else:
    print("⚠ Tidak ada data untuk di-export.")


⚠ Gagal export ke Excel. Install openpyxl dengan: pip install openpyxl
✓ Data berhasil di-export ke CSV: laporan_absensi_20251129_194510.csv


---

## E. UTILITY FUNCTIONS

### Lihat Daftar Wajah Terdaftar


In [27]:
def lihat_wajah_terdaftar():
    """
    Menampilkan daftar semua wajah yang sudah terdaftar
    """
    encoding_files = [f for f in os.listdir('encodings') if f.endswith('.pickle')]
    
    print("=" * 60)
    print("DAFTAR WAJAH TERDAFTAR")
    print("=" * 60)
    
    if len(encoding_files) == 0:
        print("\n⚠ Belum ada wajah yang terdaftar.")
        print("Silakan registrasi wajah terlebih dahulu.\n")
        return
    
    print(f"\nTotal: {len(encoding_files)} wajah terdaftar\n")
    
    data_list = []
    for i, filename in enumerate(encoding_files, 1):
        filepath = os.path.join('encodings', filename)
        with open(filepath, 'rb') as f:
            data = pickle.load(f)
            data_list.append({
                'No': i,
                'Nama': data['nama'],
                'ID': data['id'],
                'File': filename
            })
    
    df_terdaftar = pd.DataFrame(data_list)
    display(df_terdaftar)
    print("\n" + "=" * 60)

# Jalankan fungsi
lihat_wajah_terdaftar()


DAFTAR WAJAH TERDAFTAR

Total: 2 wajah terdaftar



,No,Nama,ID,File
0,1,Fahren Andrean Rangkuti,23215030,Fahren Andrean Rangkuti_23215030.pickle
1,2,Fahren,23215030,Fahren_23215030.pickle


### Reset Data Absensi (Hati-hati!)


In [7]:
def reset_absensi():
    """
    Reset/hapus semua data absensi
    HATI-HATI: Ini akan menghapus semua record absensi!
    """
    konfirmasi = input("Apakah Anda yakin ingin menghapus semua data absensi? (ketik 'HAPUS' untuk konfirmasi): ")
    
    if konfirmasi == "HAPUS":
        # Backup data lama
        if os.path.exists('absensi.csv'):
            backup_name = f"absensi_backup_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
            os.rename('absensi.csv', backup_name)
            print(f"✓ Backup data lama disimpan: {backup_name}")
        
        # Buat file baru
        df = pd.DataFrame(columns=['nama', 'id', 'waktu'])
        df.to_csv('absensi.csv', index=False)
        print("✓ Data absensi berhasil direset!")
    else:
        print("❌ Reset dibatalkan.")

# Uncomment baris di bawah jika ingin reset
reset_absensi()


Apakah Anda yakin ingin menghapus semua data absensi? (ketik 'HAPUS' untuk konfirmasi):  HAPUS


✓ Backup data lama disimpan: absensi_backup_20251129_171425.csv
✓ Data absensi berhasil direset!


---

## SELESAI!

### Panduan Penggunaan:

1. **Registrasi Wajah Baru:**
   - Jalankan cell di bagian "B. MODUL REGISTRASI WAJAH"
   - Input nama dan ID
   - Posisikan wajah di depan kamera
   - Tekan 'c' untuk capture

2. **Mulai Absensi:**
   - Jalankan cell di bagian "C. MODUL ABSENSI WAJAH"
   - Sistem akan otomatis mendeteksi dan mencatat absensi
   - Tekan 'q' untuk keluar

3. **Lihat Data:**
   - Jalankan cell di bagian "D. VISUALISASI DATA ABSENSI"
   - Lihat tabel dan statistik absensi

### Tips:
- Pastikan pencahayaan cukup saat registrasi dan absensi
- Posisikan wajah menghadap langsung ke kamera
- Jangan terlalu jauh atau terlalu dekat dari kamera
- Sistem dapat mendeteksi beberapa wajah sekaligus

### Troubleshooting:
- **Kamera tidak berfungsi:** Periksa apakah kamera digunakan aplikasi lain
- **Wajah tidak terdeteksi:** Perbaiki pencahayaan dan posisi wajah
- **Salah mengenali wajah:** Registrasi ulang dengan foto lebih jelas

---

**Sistem Smart Absensi Berbasis Wajah siap digunakan!** 🎉
